This notebook is intended to support a brief data exploration of the results from the simulations performed by using the RC models contained in script dc_ro_model.py . 

A more in depth analysis of such simulations, including a comparative one of different case studies, is available in notebook compare_scenarios.ipynb

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from pathlib import Path
PROJECT_DIR =Path(os.path.abspath('')).parents[1]

sys.path.append(os.fspath(PROJECT_DIR))
from pipeline.definitions import *

from os import listdir
from os.path import isfile, join


import matplotlib as mpl
import colorsys
import matplotlib.colors as mc
import itertools

### Select graphic settings

In [ ]:
graph_setting="notebook" #or "article"

In [ ]:
if graph_setting=="article":
    
    #journal-quality parameter settings
    resolution_factor=2
    desired_font=10

elif graph_setting=="notebook":
    resolution_factor=1
    desired_font=12
    
#conversion factors
cm_to_inch=0.393701
classic_proportion=6.4/4.8
golden_rate=1.618

#Elsevier column width is 8.4 cm, double-column width is 17.7 cm (in inches: 3.31 and 6.97)
small_figsize=(resolution_factor*3.31, resolution_factor*3.31/classic_proportion)
big_figsize=(resolution_factor*6.97, resolution_factor*6.97/classic_proportion)

#changings regarding fonttypex
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = "Arial"

font_size=resolution_factor*desired_font

#define colors palette
#thermal fluxes
colors={}
colors["Q_gain"]="orangered" 
colors["Q_sol"]="gold"
colors["Q_tras"]="mediumseagreen"
colors["Q_vent"]="plum"
colors["Q_cool"]="cornflowerblue"
#electrical fluxes
colors["P_tlc"]="orangered" 
colors["P_diss"]="crimson"
colors["P_vent"]="plum"
colors["P_cool"]="cornflowerblue"
colors["P"]="gray"

#define path for figures
figures_path=FIGURES
#check existance of figure path
if not os.path.exists(figures_path):
    print("The selected directory to store figures does not exist")

### Load data

In [ ]:
#then  load and modify simulated data to be compliant with real ones
path=SIM_DATA_NO_FORMAT
files= [f for f in listdir(path) if isfile(join(path, f))]

In [ ]:
files

In [ ]:
file='simData_building_A_cold_weather_retrofit_A'
gen_df=pd.read_csv(path+"/"+file)

### Check COP curve

In [ ]:
fig=plt.figure(figsize=(4.5,3))
#plt.scatter(df.T_ext, df.cop)
df=gen_df.sort_values(by="T_ext")
df['mean_cop'] =df.cop.rolling(10, min_periods=1).mean()
plt.vlines([10.5, 29], 0, 5, color="black", linestyle="--")
plt.axvspan(0, 10.5, alpha=0.15, color='blue')
plt.axvspan( 10.5, 29, alpha=0.15, color='green')
plt.axvspan(29, 100, alpha=0.15, color='red')
plt.scatter(df.T_ext, df['cop'], s=0.8, color="gray", alpha=0.8)
plt.ylim([0.8, 5])
plt.xlim([5, 35])
plt.yticks(np.linspace(1, 5, 9), labels=np.linspace(1, 5, 9))

plt.ylabel("COP value")
plt.xlabel("External Temperature [$T_{ext}$]")
fig.savefig(figures_path+"/simulated_cop_curve", bbox_inches='tight', dpi=200)

### Check noise addition

In [ ]:
fig, ax=plt.subplots(1, figsize=big_figsize)
#plt.plot(gen_df.P)

ax.tick_params(labelsize=font_size-2)
ax.plot(gen_df.P, color="lightcoral")
ax.plot(gen_df.P-gen_df.epsilon, color="midnightblue")

# plt.scatter(gen_df.T_ext, gen_df.P-gen_df.epsilon-gen_df.epsilon_h-gen_df.epsilon_ar-gen_df.eta, s=1)
# plt.scatter(gen_df.T_ext, gen_df.P, s=1)
ax.set_xlim([5000, 5000+24*4])
ax.set_xlabel("Time $[h]$", fontsize=font_size)
ax.set_ylabel("Electrical Load $[kWh]$", fontsize=font_size)

ax.legend(["Noisy TS","Simulated TS"], fontsize=font_size)
fig.savefig(figures_path+"/ts_noise_addition_ex", bbox_inches='tight', dpi=200)

### Check thermal fluxes

In [ ]:
df=gen_df

fig, axs = plt.subplots(3, figsize=big_figsize, sharex=True)

for ax, quota in zip(axs, ['Q_tras', 'Q_vent', 'Q_sol']):
    df[quota].plot(ax=ax,alpha=0.8, color=colors[quota])
    xlims=ax.get_xlim()
    ax.hlines(0, -1000, 19000, color="gray", linestyle="--")
    ax.set_xlim(xlims)


for ax in axs:
    ax.tick_params(labelsize=font_size-2)
    legend_labels=ax.get_legend_handles_labels()
    new_legend_labels=["$"+label.split("_")[0]+"_{"+label.split("_")[1]+"}$" for label in legend_labels[1]]
    ax.legend(labels=new_legend_labels, fontsize=font_size)

axs[1].set_ylabel('Heat Flows [W]', fontsize=font_size)
axs[2].set_xlabel("Time $[h]$", fontsize=font_size)
plt.tight_layout()
fig.savefig(figures_path+"/sim_heat_flow_ts_separated", bbox_inches='tight', dpi=200)

In [ ]:
df=gen_df


fig, axs = plt.subplots(2, figsize=big_figsize, sharex=True)

for quota in ['Q_tras', 'Q_gain', 'Q_sol', 'Q_vent']:
    df[quota].plot(ax=axs[0],alpha=0.8, color=colors[quota])
    
axs[0].set_ylabel('Heat Flows [W]', fontsize=font_size)
df['Q_cool'].plot(ax=axs[1], label='Q_cool', fontsize=font_size, color=colors["Q_cool"])
axs[1].set_ylabel('Total Heat Flow [W]', fontsize=font_size)


for ax in axs:
    ax.tick_params(labelsize=font_size-2)
    legend_labels=ax.get_legend_handles_labels()
    new_legend_labels=["$"+label.split("_")[0]+"_{"+label.split("_")[1]+"}$" for label in legend_labels[1]]
    ax.legend(labels=new_legend_labels, fontsize=font_size)

axs[1].set_xlabel("Time $[h]$", fontsize=font_size)
plt.tight_layout()
fig.savefig(figures_path+"/sim_heat_flows_ts_sum", bbox_inches='tight', dpi=200)

### Check energy balance

In [ ]:
fig, axs = plt.subplots(2, figsize=big_figsize, sharex=True)

for quota in ['P_cool', 'P_tlc', 'P_diss', 'P_vent']:
    df[quota].plot(ax=axs[0], color=colors[quota])
axs[0].set_ylabel('Electrical Power [W]', fontsize=font_size)
df['P'].plot(ax=axs[1], label='P_tot', color=colors["P"])
axs[1].set_ylabel('Total Electrical Power [W]', fontsize=font_size)

for ax in axs:
    ax.tick_params(labelsize=font_size-2)
    legend_labels=ax.get_legend_handles_labels()
    new_legend_labels=["$"+label.split("_")[0]+"_{"+label.split("_")[1]+"}$" for label in legend_labels[1]]
    ax.legend(labels=new_legend_labels, fontsize=font_size)

axs[1].set_xlabel("Time $[h]$", fontsize=font_size)
plt.tight_layout()
#fig.savefig(figures_path+"/sim_electrical_quotas_ts", bbox_inches='tight', dpi=200)